<a href="https://colab.research.google.com/github/Taghr66d/MSc-SummerProject2025/blob/main/Cusp_Samples_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pycbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [4]:
import numpy as np
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.noise import noise_from_psd
from numpy.fft import fft, fftfreq
from scipy.interpolate import interp1d

##### Cusp signal function:
def cusp_signal(amplitude=1.0, f_high=512, delta_t=1/8192, duration=0.1):
    fs = 1 / delta_t
    N = int(duration * fs)
    freqs = fftfreq(N, d=delta_t)
    h_f = np.zeros(N, dtype=complex)
    pos_mask = freqs > 0
    f_pos = freqs[pos_mask]
    spectrum = f_pos ** (-4/3)
    spectrum[f_pos >= f_high] *= np.exp(1 - f_pos[f_pos >= f_high] / f_high)
    h_f[pos_mask] = amplitude * spectrum
    h_f = h_f + np.conj(h_f[::-1])
    h_f *= np.exp(-2j * np.pi * freqs * (duration / 2))
    h_t = np.fft.ifft(h_f).real
    return h_t

###### Standard SNR computation:
def compute_standard_snr(signal, delta_t, psd, psd_freqs):
    N = len(signal)
    df = 1.0 / (N * delta_t)
    freqs = fftfreq(N, delta_t)
    signal_f = fft(signal)
    mask = freqs > 0
    freqs = freqs[mask]
    signal_f = signal_f[mask]
    psd_interp = interp1d(psd_freqs, psd, bounds_error=False, fill_value="extrapolate")
    psd_vals = psd_interp(freqs)
    psd_vals = np.maximum(psd_vals, 1e-40)
    snr_squared = 4 * np.sum((np.abs(signal_f) ** 2) / psd_vals) * df
    return np.sqrt(snr_squared)

###### Configuration:
sample_rate = 8192
delta_t = 1 / sample_rate
duration = 1.0
samples = int(sample_rate * duration)
num_samples = 6000

###### PSD used for noise + SNR computation:
psd = aLIGOZeroDetHighPower(samples // 2 + 1, delta_f=1.0 / duration, low_freq_cutoff=20)
psd_freqs = np.linspace(0, sample_rate / 2, len(psd))

###### Dataset generation
X = []
y = []

for i in range(num_samples):
    noise = noise_from_psd(samples, delta_t, psd)
    inject_signal = (i % 2 == 0)  # alternate: even index = cusp

    if inject_signal:
        snr_target = np.random.uniform(5, 20)
        f_high = np.random.uniform(50, 512)
        signal = cusp_signal(amplitude=1.0, f_high=f_high, delta_t=delta_t, duration=0.1)
        snr_measured = compute_standard_snr(signal, delta_t, psd, psd_freqs)
        signal *= snr_target / snr_measured
        start = (samples - len(signal)) // 2
        noise[start:start + len(signal)] += signal
        label = 1
    else:
        label = 0

    ### Normalize strain
    strain = (noise - np.mean(noise)) / np.std(noise)
    X.append(strain)
    y.append(label)

##### Finalize
X = np.array(X)
y = np.array(y)

print(" Dataset generated!")
print("X shape:", X.shape)  # (6000, 8192)
print("y shape:", y.shape)

##### Save to disk
np.save("X_cusp_dataset.npy", X)
np.save("y_cusp_labels.npy", y)
print(" Files saved as 'X_cusp_dataset.npy' and 'y_cusp_labels.npy'")


/usr/local/lib/python3.11/dist-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


 Dataset generated!
X shape: (6000, 8192)
y shape: (6000,)
 Files saved as 'X_cusp_dataset.npy' and 'y_cusp_labels.npy'
